# Converting and updating the main trait database

Last Updated: 2022-04-14  
Quang Nguyen  

This notebook was obtained using a conda environment with `r-base=4.1.2` and packages managed via `renv`

The role of this script is to take raw tables from Madin et al. and Weissman et al. and merge them. Additionally, include the updated version of the GOLD data set as well. 

We're prepping the latest database for export for evaluation. For this manuscript, we're merging a couple of existing databases:  

1. The comprehensive synthesis of trait-database from [Madin et al. 2020](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7275036/). This database was last updated in 2020. Most of the database's sources are static sources, with the exception of the [GOLD database](https://gold.jgi.doe.gov/downloads). As such, we're merging the existing release of the Madin et al. database with the most recent GOLD release (2022-03-12).  
2. Manual curation of bergey's manual by [Weissman et al.](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-021-04216-2). This database contains manual curation of the Bergey's manual specific to human-associated microbiomes.   

The way we're trying to combine these disparate sources would be to perform something similar to Madin et al. using the [R code](https://github.com/bacteria-archaea-traits/bacteria-archaea-traits/blob/master/R/functions.R) on GitHub. We're going to apply relevant transformations and mappings where apply.  

For all large files, please create a directory called `large_data` first before continuing.  

In [42]:
library(data.table)
library(dtplyr)
library(here)
library(stringdist)
library(tidyverse)
here::i_am("notebooks/db_prep.ipynb");
setDTthreads(4)

here() starts at /dartfs-hpc/rc/home/k/f00345k/research/microbe_set_trait



In [3]:
library(taxizedb)
db_download_ncbi(overwrite = FALSE)

Registered S3 method overwritten by 'hoardr':
  method           from
  print.cache_info httr

Database already exists, returning old file



[1] "~/.cache/R/taxizedb/NCBI.sql"

This code is used to upload data onto GitHub using `piggyback` R package
```r
piggyback::pb_upload(file = here("large_files", "goldData.xlsx"), tag = "0.1", overwrite = TRUE)
```

# Analysis

In [4]:
base <- read_csv(here("data", "condensed_species_NCBI.txt")) %>% 
    select(species_tax_id, superkingdom, phylum, class, order, family, 
           genus, species, metabolism, gram_stain, pathways, 
           carbon_substrates, sporulation, motility, cell_shape) %>% 
    rename("substrate" = carbon_substrates)

Rows: 14893 Columns: 79
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (19): species, genus, family, order, class, phylum, superkingdom, gram_s...
dbl (60): species_tax_id, d1_lo, d1_up, d2_lo, d2_up, doubling_h, genome_siz...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## GOLD and Weissman data processing

Before we merge our data sets, we have to prep all each of the individual GOLD and Weissman et al. database to unify both approaches. Check out the notebooks `notebook/gold_proc.ipynb` and `notebook/weissman_proc.ipynb`. 

In [13]:
gold <- readRDS(file = here("output", "databases", "gold_proc.rds"))
weissman <- readRDS(file = here("output", "databases", "weissman_proc.rds"))

## Checking for duplicates

First, we define a function to check for similar sounding names across all the unique pathways and substrates for all the data sets. Here, we use the `stringdist` function from the `stringdist` package. We use the standard OSA metric (also called the Damerau-Levenshtein distance) to query for potential similarly sounding names of identical pathways or compounds. 

In [14]:
check_matches <- function(df, type=c("pathways","substrates")){
    b_val <- base %>% pull(!!type) %>% unique() %>% str_split(pattern = ", ") %>%
        unlist() %>% unique() %>% na.omit() %>% as.vector()
    
    q_val <- df %>% pull(!!type) %>% unique() %>% str_split(pattern = ", ") %>% 
        unlist() %>% unique() %>% na.omit() %>% as.vector()
    
    check <- map(q_val, ~{
        match <- stringdist(a = .x, b = b_val)
        # match 0 is the same, and match > 2 is too different 
        ret <- b_val[match > 0 & match <= 2]
        if (length(ret) == 0){
            return(NA)
        } else {
            out <- tibble(
                query = rep(.x, length(ret)),
                ref = ret
            )
        }
    })
    check <- check[!sapply(check, function(x) all(is.na(x)))]
    
    return(check)
}

In [15]:
Reduce(check_matches(weissman, "pathways"), f = rbind)
Reduce(check_matches(weissman, "substrate"), f = rbind)

query,ref
<chr>,<chr>
elastin_degradation,plastic_degradation
elastin_degradation,gelatin_degradation


query,ref
<chr>,<chr>
sucrose,fucose
butanol,2-butanol
butanol,1-butanol
ethanol,methanol
glucose,fucose
mannose,maltose
fructose,fucose
lactose,lactate
lactose,galactose


We can see that for a lot of the compounds the names might be the same but they're actually different. However, there are certain conventions such as "_" for spaces or "-" that we might need to address for the final merge.  

Let's check the GOLD database for similar sounding names. Since GOLD does not have substrate information, we only check for pathways

In [17]:
Reduce(check_matches(gold, "pathways"), f = rbind)

query,ref
<chr>,<chr>
Nitrogen fixation,nitrogen_fixation
Methane oxidation,methane_oxidation
Chitin degradation,chitin_degradation
Chitin_degradation,chitin_degradation
Nitrogen_fixation,nitrogen_fixation
Methane_oxidation,methane_oxidation


In [18]:
trim_path <- function(vec){
    vec <- vec %>% tolower() %>% 
        str_split(pattern = "(, |\\|)") %>% 
        map(~{
            str_trim(.x) %>% str_replace_all("\\-", "") %>%
                str_replace_all(" ", "_") %>% unique()
        })
    return(vec)
}


gold_final$pathways <- trim_path(gold_final$pathways)
base$pathways <- trim_path(base$pathways)
weissman$pathways <- trim_path(weissman$pathways)
weissman$substrate <- trim_path(weissman$substrate)
base$substrate <- trim_path(base$substrate)

## Combine all data frames

After munging, let's combine all of the names! The strategy is very similar to handling multiple entries for GOLD. First, we bind all of our databases together. Then, we `group_by` and nest all our trait data into a list. Then we process these lists and return `unique` rows (deduplicated). If the rows are not unique, then we process the non-unique rows by either concatenating the traits together or vote on consensus using the top most represented trait.  

In [21]:
complete <- bind_rows(
    base %>% mutate(substrate = map_chr(substrate, ~{paste0(.x, collapse = ",")}), 
                    pathways = map_chr(pathways, ~{paste0(.x, collapse = ",")})) %>% 
            mutate(source = "madin"), 
    gold %>% mutate(pathways = map_chr(pathways, ~{paste0(.x, collapse = ",")})) %>% 
            mutate(source = "gold"), 
    weissman %>% mutate(substrate = map_chr(substrate, ~{paste0(.x, collapse = ",")}), 
                        pathways = map_chr(pathways, ~{paste0(.x, collapse = ",")})) %>% 
            mutate(source = "weissman")
)
complete <- complete %>% filter(!is.na(species)) %>% 
    group_by(species_tax_id) %>%
    nest(names = c(superkingdom, phylum, class, order, family, genus, species), 
         data = c(metabolism, gram_stain, pathways, substrate, sporulation, motility, cell_shape, source)) %>%
    ungroup() %>%
    mutate(row_data = map_dbl(data, ~{nrow(.x)}))

In [22]:
head(complete)

species_tax_id,names,data,row_data
<dbl>,<list>,<list>,<dbl>
1243001,"Bacteria , Bacteria , Actinobacteria , Actinobacteria , Actinobacteria , Actinomycetia , Propionibacteriales , Propionibacteriales , Propionibacteriaceae , Propionibacteriaceae , Acidipropionibacterium , Acidipropionibacterium , Acidipropionibacterium damnosum, Acidipropionibacterium damnosum","microaerophilic, microaerophilic, NA , positive , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1679466,"Bacteria , Bacteria , Bacteroidetes , Bacteroidetes , Flavobacteriia , Flavobacteriia , Flavobacteriales , Flavobacteriales , Flavobacteriaceae , Weeksellaceae , Apibacter , Apibacter , Apibacter adventoris, Apibacter adventoris","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1591092,"Bacteria , Proteobacteria , Betaproteobacteria, Neisseriales , Chromobacteriaceae, Aquaspirillum , Aquaspirillum soli","microaerophilic, NA , NA , NA , NA , NA , NA , madin",1
1904463,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria , Epsilonproteobacteria , Campylobacterales , Campylobacterales , Campylobacteraceae , Arcobacteraceae , Arcobacter , Poseidonibacter , Arcobacter lekithochrous , Poseidonibacter lekithochrous","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1935204,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria, Epsilonproteobacteria, Campylobacterales , Campylobacterales , Campylobacteraceae , Arcobacteraceae , Arcobacter , Aliarcobacter , Arcobacter porcinus , [Arcobacter] porcinus","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
209458,"Bacteria , Firmicutes , Bacilli , Bacillales , Bacillaceae , Bacillus , Bacillus arbutinivorans","microaerophilic, NA , NA , NA , NA , NA , NA , madin",1


In [23]:
multiple_rows <- complete %>% filter(row_data >= 2) %>% pull(species_tax_id)
length(multiple_rows)

[1] 12342

In [53]:
reconcile <- complete %>% filter(species_tax_id %in% multiple_rows)

In [54]:
head(reconcile)

species_tax_id,names,data,row_data
<dbl>,<list>,<list>,<dbl>
1243001,"Bacteria , Bacteria , Actinobacteria , Actinobacteria , Actinobacteria , Actinomycetia , Propionibacteriales , Propionibacteriales , Propionibacteriaceae , Propionibacteriaceae , Acidipropionibacterium , Acidipropionibacterium , Acidipropionibacterium damnosum, Acidipropionibacterium damnosum","microaerophilic, microaerophilic, NA , positive , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1679466,"Bacteria , Bacteria , Bacteroidetes , Bacteroidetes , Flavobacteriia , Flavobacteriia , Flavobacteriales , Flavobacteriales , Flavobacteriaceae , Weeksellaceae , Apibacter , Apibacter , Apibacter adventoris, Apibacter adventoris","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1904463,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria , Epsilonproteobacteria , Campylobacterales , Campylobacterales , Campylobacteraceae , Arcobacteraceae , Arcobacter , Poseidonibacter , Arcobacter lekithochrous , Poseidonibacter lekithochrous","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1935204,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria, Epsilonproteobacteria, Campylobacterales , Campylobacterales , Campylobacteraceae , Arcobacteraceae , Arcobacter , Aliarcobacter , Arcobacter porcinus , [Arcobacter] porcinus","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1780362,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria , Epsilonproteobacteria , Campylobacterales , Campylobacterales , Campylobacteraceae , Campylobacteraceae , Campylobacter , Campylobacter , Campylobacter geochelonis, Campylobacter geochelonis","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2
1848766,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria , Epsilonproteobacteria , Campylobacterales , Campylobacterales , Campylobacteraceae , Campylobacteraceae , Campylobacter , Campylobacter , Campylobacter ornithocola, Campylobacter ornithocola","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2


Let's collapse traits of the same species similar to before and merge everything together to save to a database. Since Weissman et al. is a manually curated source, we're going to prefer Weissman et al. above other sources if there are conflicts. The second preference would be Madin et al. since Madin et al. also has curated static sources while GOLD relies on community submissions

In [55]:
#' @param df This is a data frame of multiple columns, where the columns of 
#'     pathways and substrates are themselves lists 
reconcile_trait <- function(df, index){
    nonlist <- c("metabolism", "gram_stain", "sporulation", "motility", 
                 "cell_shape")    
    # reconcile each non-pathway-substrate trait 
    out <- suppressMessages(map_dfc(nonlist, ~{
        traits <- df %>% pull(.x)        
        traits <- as.data.frame(table(traits))
        if (nrow(traits) == 1){
            return(df %>% filter(!!as.symbol(.x) == traits$traits) %>% pull(.x) %>% unique())
        } else if (nrow(traits) == 0){
            return(NA_character_)
        } else {
            w_source <- df %>% filter(source == "weissman") %>% pull(.x) 
            w_source <- na.omit(unique(w_source))
            if (length(w_source) >= 2) {
                print(index)
            }
            if (is_empty(w_source)){
                return(df %>% filter(source == "madin") %>% pull(.x))
            } else if (is.na(w_source)) {
                return(df %>% filter(source == "madin") %>% pull(.x))
            } else {
                return(w_source)
            }
        }
    }))
    names(out) <- nonlist
    # process pathway and substrate traits
    path_vec <- df %>% pull("pathways") %>% 
        map(., ~{ str_split(.x, ',')[[1]] }) %>% 
        Reduce(f = c, x = .) %>% unique()
    path_vec <- path_vec[!path_vec %in% c("NA", NA_character_)]
    if (length(path_vec) == 0){
        out$pathways <- NA_character_
    } else {
        out$pathways <- paste(path_vec, collapse = ",")
    }
    
    path_substr <- df %>% pull("substrate") %>% 
        map(., ~{ str_split(.x, ',')[[1]] }) %>% 
        Reduce(f = c, x = .) %>% unique()
    path_substr <- path_substr[!path_substr %in% c("NA", NA_character_)]
    if (length(path_substr) == 0){
        out$substr <- NA_character_
    } else {
        out$substr <- paste(path_substr, collapse = ",")
    }
    return(out)
}

In [58]:
nonlist <- c("metabolism", "gram_stain", "sporulation", "motility", 
             "cell_shape") 

reconcile[2055,]$data[[1]] %>% filter(source == "weissman")

metabolism,gram_stain,pathways,substrate,sporulation,motility,cell_shape,source
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
facultative anaerobic,NA,"catalase,oxidase,urease,esculin_aesculin_hydrolysis","arabinose,citrate,fructose,galactose,gluconate,glycerol,lactate,malate,mannose,myo_inositol,propionate,pyruvate,rhamnose,succinate",NA,no,bacillus,weissman
obligate aerobic,NA,"catalase,oxidase,esculin_aesculin_hydrolysis,hydrogen_sulfide_synthesis","arabinose,citrate,fructose,galactose,glucose,salicin,xylose",NA,no,NA,weissman


In [52]:
reconcile <- reconcile %>% mutate(traits = imap(data, ~{
    reconcile_trait(.x, .y)
}));

[1] 2055


Warning message in if (is.na(w_source)) {:
“the condition has length > 1 and only the first element will be used”


[1] 2128


Warning message in if (is.na(w_source)) {:
“the condition has length > 1 and only the first element will be used”


[1] 2572


...

Warning message in if (is.na(w_source)) {:
“the condition has length > 1 and only the first element will be used”


[1] 6708


Warning message in if (is.na(w_source)) {:
“the condition has length > 1 and only the first element will be used”


[1] 8554


Warning message in if (is.na(w_source)) {:
“the condition has length > 1 and only the first element will be used”


In [402]:
dim(reconcile)
head(reconcile)

[1] 12342     5

species_tax_id,names,data,row_data,traits
<dbl>,<list>,<list>,<dbl>,<list>
1243001,"Bacteria , Bacteria , Actinobacteria , Actinobacteria , Actinobacteria , Actinomycetia , Propionibacteriales , Propionibacteriales , Propionibacteriaceae , Propionibacteriaceae , Acidipropionibacterium , Acidipropionibacterium , Acidipropionibacterium damnosum, Acidipropionibacterium damnosum","microaerophilic, microaerophilic, NA , positive , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2,"microaerophilic, positive , NA , NA , NA , NA , NA"
1679466,"Bacteria , Bacteria , Bacteroidetes , Bacteroidetes , Flavobacteriia , Flavobacteriia , Flavobacteriales , Flavobacteriales , Flavobacteriaceae , Weeksellaceae , Apibacter , Apibacter , Apibacter adventoris, Apibacter adventoris","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2,"microaerophilic, NA , NA , NA , NA , NA , NA"
1904463,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria , Epsilonproteobacteria , Campylobacterales , Campylobacterales , Campylobacteraceae , Arcobacteraceae , Arcobacter , Poseidonibacter , Arcobacter lekithochrous , Poseidonibacter lekithochrous","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2,"microaerophilic, NA , NA , NA , NA , NA , NA"
1935204,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria, Epsilonproteobacteria, Campylobacterales , Campylobacterales , Campylobacteraceae , Arcobacteraceae , Arcobacter , Aliarcobacter , Arcobacter porcinus , [Arcobacter] porcinus","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2,"microaerophilic, NA , NA , NA , NA , NA , NA"
1780362,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria , Epsilonproteobacteria , Campylobacterales , Campylobacterales , Campylobacteraceae , Campylobacteraceae , Campylobacter , Campylobacter , Campylobacter geochelonis, Campylobacter geochelonis","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2,"microaerophilic, NA , NA , NA , NA , NA , NA"
1848766,"Bacteria , Bacteria , Proteobacteria , Proteobacteria , Epsilonproteobacteria , Epsilonproteobacteria , Campylobacterales , Campylobacterales , Campylobacteraceae , Campylobacteraceae , Campylobacter , Campylobacter , Campylobacter ornithocola, Campylobacter ornithocola","microaerophilic, NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , NA , madin , gold",2,"microaerophilic, NA , NA , NA , NA , NA , NA"


In [69]:
dim(reconcile)
head(reconcile)
# check for multiple rows for reconcile
reconcile %>% group_by(species_tax_id) %>% tally() %>% filter(n > 1)

[1] 7476    9

species_tax_id,superkingdom,phylum,class,order,family,genus,species,traits
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<list>
1780362,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter geochelonis,"microaerophilic, NA , NA , NA , NA , NA , NA"
1848766,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter ornithocola,"microaerophilic, NA , NA , NA , NA , NA , NA"
654363,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,Campylobacter troglodytis,"microaerophilic, NA , NA , NA , NA , NA , NA"
247486,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Comamonadaceae,Curvibacter,Curvibacter fontanus,"microaerophilic, NA , NA , NA , NA , NA , NA"
1453587,Bacteria,Firmicutes,Bacilli,Lactobacillales,Enterococcaceae,Enterococcus,Enterococcus crotali,"microaerophilic, positive , NA , NA , NA , NA , NA"
1159752,Bacteria,Firmicutes,Bacilli,Lactobacillales,Enterococcaceae,Enterococcus,Enterococcus lemanii,"microaerophilic, NA , NA , NA , NA , NA , NA"


species_tax_id,superkingdom,phylum,class,order,family,genus,species,n
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>


Even though we have a nice set of traits and associated NCBIids, we also want to make sure the names are up-to-date. 

In [ ]:
#' This function takes an NCBI identifier, run `classification` through it (from taxizedb) 
#' and retrieve all the ranks from superkingdom to species
#' @param identifier A string representing NCBIids
#' @param names A data.frame representing one or multiple hypothetical names. 
#'     for use mostly when NCBIids do not resolve
collapse_name <- function(identifier, names) {
    names <- names %>% dplyr::distinct() 
    query_ranks <- classification(identifer, db = "ncbi", verbose = FALSE)[[1]]
    # if query via identifier does not work. 
    # do this reverse query thing where the name is re-queried back to get NCBIids
    if (nrow(query_ranks) == 0){
        cand_ids = vector(length = nrow(names))
        for (i in seq_len(nrow(names))){
            # rev_query here is a data frame
            rev_query <- name2taxid(names$species, out_type = "summary")
            if (nrow(rev_query) != 1){
                # if there are ambiguous names or if there are no matches, return NA
                cand_ids[i] <- NA_character_
            } else {
                cand_ids[i] <- rev_query$id
            }
        }
        # remove NAs and get only unique ids
        cand_ids <- unique(na.omit(cand_ids))
        # if each name returns a different id then return NA because it's confusing
        if (length(cand_ids) >= 2){
            # if more than one name then just concatenate all the names together 
            reconciled_names <- as_tibble(map(names, ~{ paste0(.x, collapse = "|") }))
        } else if (length(cand_ids) == 1) {
            reconciled_names <- classification(cand_ids, db = "ncbi")[[1]] %>% 
                filter(!rank %in% c("no rank", "clade")) %>% 
                select(-id) %>% pivot_wider(names_from = rank, values_from = name)
        } else {
            # create an empty final names
            reconciled_names <- matrix(rep(NA_character_, ncol(names)), nrow = 1, ncol = ncol(names))
            colnames(reconciled_names) <- colnames(names)
            reconciled_names <- as_tibble(reconciled_names)
        }
    } else {
        reconciled_names <- query_ranks %>% filter(!rank %in% c("no rank", "clade")) %>% select(-id) %>% 
            pivot_wider(names_from = rank, values_from = name)
    }
    return(reconciled_names)
}

We combine the original `complete`, removing all rows that have multiple rows and then combine it with the reconciled version

In [70]:
complete <- bind_rows(reconcile %>% unnest(traits), 
          complete %>% filter(!species_tax_id %in% multiple_rows))

species_tax_id,n
<dbl>,<int>
7,3
9,3
11,2
14,2
16,1
17,2
19,2
20,1
21,2


In [71]:
complete %>% filter(species_tax_id %in% multiple_rows) %>% 
    group_by(species_tax_id, superkingdom, phylum, class, order, 
             family, genus, species) %>% tally() 

species_tax_id,superkingdom,phylum,class,order,family,genus,species,n
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
9,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Erwiniaceae,Buchnera,aphidicola,1
9,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacterales,Erwiniaceae,Buchnera,Buchnera aphidicola,1
17,Bacteria,Proteobacteria,Betaproteobacteria,Nitrosomonadales,Methylophilaceae,Methylophilus,Methylophilus methylotrophus,2
21,Bacteria,Proteobacteria,Alphaproteobacteria,Caulobacterales,Caulobacteraceae,Phenylobacterium,Phenylobacterium immobile,1
23,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Shewanellaceae,Shewanella,Shewanella colwelliana,1
24,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Shewanellaceae,Shewanella,putrefaciens,1
24,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Shewanellaceae,Shewanella,Shewanella putrefaciens,1
25,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Shewanellaceae,Shewanella,Shewanella hanedai,1
33,Bacteria,Proteobacteria,Deltaproteobacteria,Myxococcales,Myxococcaceae,Myxococcus,fulvus,1


In [64]:
# check if complete still has more than one columns 
complete %>% group_by(species_tax_id, superkingdom, phylum, class, 
                      order, family, genus, species) %>% 
            tally() %>% filter(n > 1)

complete

species_tax_id,superkingdom,phylum,class,order,family,genus,species,n
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
17,Bacteria,Proteobacteria,Betaproteobacteria,Nitrosomonadales,Methylophilaceae,Methylophilus,Methylophilus methylotrophus,2
34,Bacteria,Proteobacteria,Deltaproteobacteria,Myxococcales,Myxococcaceae,Myxococcus,Myxococcus xanthus,2
38,Bacteria,Proteobacteria,Deltaproteobacteria,Myxococcales,Archangiaceae,Archangium,Archangium disciforme,2
52,Bacteria,Proteobacteria,Deltaproteobacteria,Myxococcales,Polyangiaceae,Chondromyces,Chondromyces crocatus,2
139,Bacteria,Spirochaetes,Spirochaetia,Spirochaetales,Borreliaceae,Borreliella,Borreliella burgdorferi,2
147,Bacteria,Spirochaetes,Spirochaetia,Spirochaetales,Spirochaetaceae,Spirochaeta,Spirochaeta aurantia,2
150,Bacteria,Spirochaetes,Spirochaetia,Spirochaetales,Spirochaetaceae,Spirochaeta,Spirochaeta isovalerica,2
180,Bacteria,Nitrospirae,Nitrospira,Nitrospirales,Nitrospiraceae,Leptospirillum,Leptospirillum ferrooxidans,2
188,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,Rhodospirillaceae,Magnetospirillum,Magnetospirillum magnetotacticum,2


In [ ]:
saveRDS(complete, file = here("output", "db_merged.rds"))
complete %>% mutate(
    pathways = map_chr(pathways, ~{
        paste(.x, collapse = ", ")
    }), 
    substrate = map_chr(substrate, ~{
        paste(.x, collapse = ", ")
    })
) %>% write_csv(x = ., file = here("output", "db_merged.csv"))